In [6]:
from theano.sandbox import cuda
# cuda.use('gpu2')

In [7]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [8]:
batch_size = 64
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

14909440/15296311 [============================>.] - ETA: 0s

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [9]:
X_test = np.expand_dims(X_test, 1)
X_train = np.expand_dims(X_train, 1)

X_test.shape

(10000, 1, 28, 28)

In [10]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [11]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [12]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [13]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [14]:
def norm_input(x):
    return (x-mean_px)/std_px

In [15]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [16]:
lm = get_lin_model()

In [17]:
gen = image.ImageDataGenerator()
# Generating augmented images of the dataset
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [18]:
lm.fit_generator(batches, batches.N, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.4229 - acc: 0.8744 - val_loss: 0.2948 - val_acc: 0.9179


In [19]:
lm.optimizer.lr = 0.1

In [20]:
lm.fit_generator(batches, batches.N, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.N)   

Epoch 1/1
60000/60000 [==============================] - 11s - loss: 0.2984 - acc: 0.9157 - val_loss: 0.2802 - val_acc: 0.9217


In [21]:
lm.optimizer.lr = 0.01

In [23]:
lm.fit_generator(batches, batches.N, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.2826 - acc: 0.9205 - val_loss: 0.2858 - val_acc: 0.9200


In [24]:
lm.fit_generator(batches, batches.N, nb_epoch=3, validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/3
60000/60000 [==============================] - 12s - loss: 0.2789 - acc: 0.9218 - val_loss: 0.2838 - val_acc: 0.9221
Epoch 2/3
60000/60000 [==============================] - 12s - loss: 0.2709 - acc: 0.9247 - val_loss: 0.2725 - val_acc: 0.9238
Epoch 3/3
60000/60000 [==============================] - 12s - loss: 0.2716 - acc: 0.9238 - val_loss: 0.2824 - val_acc: 0.9214


### Dense Layer Model

In [26]:
def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(512, activation='softmax'),
        Dense(10, activation='softmax')     
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [27]:
fc = get_fc_model()

In [28]:
# Grossly underfitting here BRUH

fc.fit_generator(batches, batches.N, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 1.5530 - acc: 0.8709 - val_loss: 1.0001 - val_acc: 0.9219


In [29]:
fc.optimizer.lr = 0.1

In [30]:
# Overfitting here too BRUH
fc.fit_generator(batches, batches.N, nb_epoch=4, validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 12s - loss: 0.7312 - acc: 0.9260 - val_loss: 0.5391 - val_acc: 0.9305
Epoch 2/4
60000/60000 [==============================] - 12s - loss: 0.4466 - acc: 0.9328 - val_loss: 0.3809 - val_acc: 0.9325
Epoch 3/4
60000/60000 [==============================] - 12s - loss: 0.3427 - acc: 0.9365 - val_loss: 0.3225 - val_acc: 0.9349
Epoch 4/4
60000/60000 [==============================] - 12s - loss: 0.2918 - acc: 0.9408 - val_loss: 0.2898 - val_acc: 0.9368


In [31]:
fc.optimizer.lr = 0.01

In [32]:
# Not too much change in terms of overfitting.

fc.fit_generator(batches, batches.N, nb_epoch=4, validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 12s - loss: 0.2648 - acc: 0.9431 - val_loss: 0.2819 - val_acc: 0.9348
Epoch 2/4
60000/60000 [==============================] - 12s - loss: 0.2459 - acc: 0.9455 - val_loss: 0.2767 - val_acc: 0.9346
Epoch 3/4
60000/60000 [==============================] - 12s - loss: 0.2350 - acc: 0.9461 - val_loss: 0.2500 - val_acc: 0.9399
Epoch 4/4
60000/60000 [==============================] - 12s - loss: 0.2224 - acc: 0.9481 - val_loss: 0.2531 - val_acc: 0.9368


### Analyzing MNIST using a VGG style architecture.

In [33]:
def get_model():
    model = Sequential([
            Lambda(norm_input, input_shape=(1,28,28)),
            Convolution2D(32,3,3, activation='relu'),
            Convolution2D(32,3,3, activation='relu'),
            MaxPooling2D(),
            Convolution2D(64,3,3, activation='relu'),
            Convolution2D(63,3,3, activation='relu'),
            MaxPooling2D(),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [34]:
model = get_model()

In [35]:
# Serious underfitting going on bruh
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.1057 - acc: 0.9679 - val_loss: 0.0338 - val_acc: 0.9895


In [36]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_3 (Lambda)                (None, 1, 28, 28)     0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 26, 26)    320         lambda_3[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 24, 24)    9248        convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 12, 12)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [37]:
model.optimizer.lr = 0.1

In [38]:
# Still underfitting bruh
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 20s - loss: 0.0360 - acc: 0.9881 - val_loss: 0.0279 - val_acc: 0.9914


In [39]:
model.optimizer.lr = 0.01

In [40]:
# The model fit goooooooodish, about 99.62 percent train acc vs 99.05% val_acc
# Basically, I'm overfitting, but I'm kinda happy about it.
model.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 20s - loss: 0.0242 - acc: 0.9921 - val_loss: 0.0232 - val_acc: 0.9927
Epoch 2/4
60000/60000 [==============================] - 21s - loss: 0.0181 - acc: 0.9945 - val_loss: 0.0380 - val_acc: 0.9888
Epoch 3/4
60000/60000 [==============================] - 20s - loss: 0.0163 - acc: 0.9947 - val_loss: 0.0310 - val_acc: 0.9920
Epoch 4/4
60000/60000 [==============================] - 20s - loss: 0.0121 - acc: 0.9962 - val_loss: 0.0332 - val_acc: 0.9905


In [41]:
model.optimizer.lr=0.001

In [42]:
model.fit_generator(batches, batches.N, nb_epoch=14, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/14
60000/60000 [==============================] - 21s - loss: 0.0112 - acc: 0.9963 - val_loss: 0.0256 - val_acc: 0.9929
Epoch 2/14
60000/60000 [==============================] - 20s - loss: 0.0104 - acc: 0.9967 - val_loss: 0.0232 - val_acc: 0.9932
Epoch 3/14
60000/60000 [==============================] - 20s - loss: 0.0088 - acc: 0.9970 - val_loss: 0.0361 - val_acc: 0.9914
Epoch 4/14
60000/60000 [==============================] - 20s - loss: 0.0085 - acc: 0.9972 - val_loss: 0.0304 - val_acc: 0.9940
Epoch 5/14
60000/60000 [==============================] - 20s - loss: 0.0058 - acc: 0.9982 - val_loss: 0.0413 - val_acc: 0.9908
Epoch 6/14
60000/60000 [==============================] - 20s - loss: 0.0068 - acc: 0.9979 - val_loss: 0.0327 - val_acc: 0.9920
Epoch 7/14
60000/60000 [==============================] - 20s - loss: 0.0049 - acc: 0.9984 - val_loss: 0.0280 - val_acc: 0.9940
Epoch 8/14
60000/60000 [==============================] - 21s - loss: 0.0050 - acc: 0.9985 - val_loss: 0

## Adding Batchnorm + Dropout

In [43]:
def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [44]:
model = get_model_bn_do()

In [45]:
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 47s - loss: 0.1139 - acc: 0.9660 - val_loss: 0.0411 - val_acc: 0.9884


In [46]:
model.optimizer.lr = 0.1

In [47]:
model.fit_generator(batches, batches.N, nb_epoch=4, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

Epoch 1/4
60000/60000 [==============================] - 47s - loss: 0.0452 - acc: 0.9859 - val_loss: 0.0321 - val_acc: 0.9899
Epoch 2/4
60000/60000 [==============================] - 46s - loss: 0.0372 - acc: 0.9889 - val_loss: 0.0226 - val_acc: 0.9927
Epoch 3/4
60000/60000 [==============================] - 46s - loss: 0.0307 - acc: 0.9905 - val_loss: 0.0303 - val_acc: 0.9900
Epoch 4/4
60000/60000 [==============================] - 46s - loss: 0.0286 - acc: 0.9909 - val_loss: 0.0270 - val_acc: 0.9919
